In [1]:
import pulp
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools
from copy import deepcopy as dcopy,copy
from skimage.morphology import label

In d:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In d:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In d:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
# Working through the hooks puzzles.
# Different to #2 as many more limitations on potential hook placements. 24 instead of 3200
# need to work out a good method of solving !
# Brute force backtracking works OK for the outside rows. May need to solve top down and bottom up then check he middle row.
# the connected condition is very important on the inner squares.

### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/02/hook3_puzzle-01.png" width="300" height="300">

In [3]:
#Setup the constraints
top_labels = [0,35,42,18,18,0, 36, 63,0]
bot_labels = [0,40,32,40,10,12,0,56,0]
left_labels = [0,56,0,32,40,15,16,25,0]
right_labels =[0,49,63,0,18,42,63,54,0]
params = [top_labels,bot_labels,left_labels,right_labels]

In [4]:
class Matrix():
    def __init__(self,top_labels,bot_labels,left_labels,right_labels):
        self.top_labels = top_labels
        self.bot_labels = bot_labels
        self.left_labels = left_labels
        self.right_labels = right_labels
        self.potential_grids = [[-9,np.ones((9,9),dtype=int)*-1,[0,0,9,9]]] # list of all grids not yet ruled out.[Level,grid,coordinates of the grid yet to be filled]
        self.solution = []
        self.splits =  [list(i) for i in itertools.product([0, 1], repeat=9) if sum(i) > 1]
     
    def add_layer(self,grid,coords,lvl,alignment):
        row_start,col_start,row_end, col_end = coords
    
        if alignment == 0:
            grid[row_start:row_end,col_start] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            grid[row_start:row_end,col_start] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_start +=1 

        if alignment == 2:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_end -=1 
    
        coords = [row_start,col_start,row_end, col_end]
        
        return grid,coords    
   
    def check_grid(self,grid):
        isValid = 1
        for i in range(9):
            row = grid[i,:]
            col = grid[:,i]
            if -1 not in row:
                    isValid *= self.check_line(row,self.left_labels[i],self.right_labels[i])
            if -1 not in col:
                    isValid *= self.check_line(col,self.top_labels[i],self.bot_labels[i])  
        return isValid
                    
    def check_line(self,line,start,end):
        for split in self.splits:
            test = line * split
            if (test[test!=0][:2].prod() == start) or (start ==0):
                if (test[test!=0][-2:].prod() == end) or (end ==0):
                    return 1
        return 0
    
    
    def solve_matrix(self,y,solution = np.ones((9,9),dtype = int)*-1):
        for row in range(9):
            if -1 in solution[row,:]:
                for split in self.splits:
                    if self.possible(row,split,solution,y):
                            solution[row,:] = split
                            self.solve_matrix(y,solution=solution)
                            solution[row,:] = np.ones(9,dtype=int)*-1
                return
        print("SOLVED")
        self.final = dcopy(solution),dcopy(y)
    
    def possible(self,row,split,solution,y):
        #check the line works
        
        test = y[row,:] * split
        start = self.left_labels[row]
        end = self.right_labels[row]
        if (test[test!=0][:2].prod() != start) and (start !=0):
            if (test[test!=0][-2:].prod() != end) and (end !=0):
                return False
                
        #check the columns work
        return self.column_check(row,split,solution,y)

    
    def column_check(self,row,split,solution,y):
        temp = dcopy(solution)
        temp[row,:] = split
        
        #check connectivuty
        if np.max(label(temp!=0,connectivity=1)) > 1 :
            return False
        
        temp *= y
        for i in range(9):
            col = temp[:,i]
            if np.sum(col > 0) == 1:
                if self.top_labels[i] % col[col>0]  != 0:
                    return False
            elif np.sum(col>0) > 1:
                if col[col!=0][:2].prod() != self.top_labels[i]:
                    return False
            if row ==8:
                 if col[col!=0][-2:].prod() != self.bot_labels[i]:
                    return False
        return True   
    
    
    def solve(self):
        
        correct = np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],
                    [8, 6, 6, 6, 6, 6, 6, 7, 9],
                    [8, 5, 4, 3, 2, 1, 6, 7, 9],
                    [8, 5, 4, 3, 2, 2, 6, 7, 9],
                    [8, 5, 4, 3, 3, 3, 6, 7, 9],
                    [8, 5, 4, 4, 4, 4, 6, 7, 9],
                    [8, 5, 5, 5, 5, 5, 6, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]])
            
        while len(self.potential_grids) > 0:
            
            temp_grid = self.potential_grids.pop(0)
            #create the potential rotations at the given level
            rotations = []
        
            for alignment in range(4):
                lvl,grid,coords = dcopy(temp_grid)
                grid,coords = self.add_layer(grid,coords,-lvl,alignment)
                if lvl != -1 :
                    rotations.append([lvl+1,grid,coords])
                else:
                     rotations = [[lvl+1,grid,coords]]
    
            
            #check valid grids (where the sum can be made from available digits) and save the ones that work
            for i in range(len(rotations)):
                lvl,g,coords = rotations[i]
                if self.check_grid(g):
                    if lvl !=0:    
                        self.potential_grids.append([lvl,g,coords])
                    else:
                        self.solution.append(g)
                        
        self.solution = []
        self.solution = [correct]
                        
        print("There are {} valid hooks".format(len(self.solution)))
        print("Check correct grid:",any(np.array_equal(correct, i) for i in self.solution))
        
        #solve each grid in the cut down list
        for i in range(len(self.solution)):
            grid = self.solution[i]
            #self.solve_matrix(grid)
            
                            
def sol_print(solved,matrix):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax = sns.heatmap(matrix,annot=solved*matrix,cbar=False,cmap="YlGnBu")
    ax.axis("off")
    
    

In [ ]:
start = time.perf_counter()
test = Matrix(top_labels,bot_labels,left_labels,right_labels)
test.solve()
solved,matrix = test.final
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds\n'.format((stop-start)))
#sol_print(solved,matrix)

### Puzzle solution
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/03/20180228_hooks_3_ans.png" width="300" height="400">

In [6]:
correct = np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],
                    [8, 6, 6, 6, 6, 6, 6, 7, 9],
                    [8, 5, 4, 3, 2, 1, 6, 7, 9],
                    [8, 5, 4, 3, 2, 2, 6, 7, 9],
                    [8, 5, 4, 3, 3, 3, 6, 7, 9],
                    [8, 5, 4, 4, 4, 4, 6, 7, 9],
                    [8, 5, 5, 5, 5, 5, 6, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]],dtype=int)

solved=   np.array([[0, 0, 0, 0, 9, 9, 0, 9, 0],
                    [8, 7, 7, 0, 0, 7, 0, 7, 0],
                    [0, 0, 6, 6, 0, 6, 6, 7, 9],
                    [8, 0, 4, 0, 0, 1, 0, 0, 9],
                    [8, 5, 4, 3, 2, 2, 0, 0, 9],
                    [0, 5, 0, 3, 0, 3, 6, 7, 0],
                    [0, 0, 0, 4, 0, 4, 0, 7, 9],
                    [0, 5, 0, 5, 5, 0, 6, 0, 9],                          
                    [0, 8, 8, 8, 0, 0, 8, 8, 9]],dtype=int)

incorrect=np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 6, 6, 6, 6, 6, 6, 9],
                    [8, 7, 5, 5, 5, 5, 5, 6, 9],
                    [8, 7, 4, 3, 2, 1, 5, 6, 9],
                    [8, 7, 4, 3, 2, 2, 5, 6, 9],
                    [8, 7, 4, 3, 3, 3, 5, 6, 9],
                    [8, 7, 4, 4, 4, 4, 5, 6, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]],dtype=int)

matrix = solved/correct


In [7]:
splits = [list(i) for i in itertools.product([0, 1], repeat=9) if sum(i) > 1]
 
    
def column_check_top(row,split,solution,y):
        temp = copy(solution)
        temp[row,:] = split       
        
        #check connectivuty
        if np.max(label(temp!=0,connectivity=1)) > 1 :
            #print("fail connect")
            return False
        
        temp *= y
        
        #check counts
        for j in range(1,10):
            if np.sum(temp == j) > j:
                #print(temp)
                return False
        
        # check that the top conditions are 
        for i in range(9):
            col = temp[:,i]
            if top_labels[i] != 0:
                
                if split[i] == 1:
                    previous = np.sum(col[:row] !=0)
                    num = col[row]
                    
                    if previous == 0:
                        if top_labels[i] % num !=0:
                            #print("fail divisor")
                            return False
                    
                        remaining = top_labels[i] / num
                        if remaining not in y[row+1:,i]:
                            #print("fail remaining. col = {} label = {} number= {} remaining = {} ".format(i,top_labels[i],num,remaining))
                            return False                    
                    
                    elif previous == 1:
                        if col[col!=0][:2].prod() != top_labels[i]:
                            #print("fail total")
                            return False
                else:
                    previous = np.sum(col[:row] !=0)
                    if previous == 1:
                        num = np.sum(col[:row])
                        remaining = top_labels[i] / num
                        if remaining not in y[row+1:,i]:
                            return False     
                    elif previous == 0:
                        check = 0
                        
                        if not any(n in y[row+1:,i] for n in (np.ones(8-row,dtype=int)*top_labels[i]) / y[row+1:,i]):
                                #print(np.ones(8-row)*top_labels[i] / y[row+1:,i],y[row+1:,i])
                                return False
        return True  
    
def possible(row,split,solution,y):
        #check the line works
        test = y[row,:] * split
        start = left_labels[row]
        end = right_labels[row]
        
        if 1 in y[row]:
            if 1 not in test:
                return False
        
        if (end !=0) or (start !=0):
            rem_zero = test[test!=0]
            if (rem_zero [:2].prod() != start) and (start !=0):
                return False      
            
            if (rem_zero [-2:].prod() != end) and (end !=0):
                return False
        return True   

# top down solvers 1-4

In [8]:
start = time.perf_counter()
first_row =[]

solution = np.ones((9,9),dtype=int)*-1

for split in splits: 
    if possible(0,split,solution,correct):
        if column_check_top(0,split,solution,correct):
            first_row.append(split)
            
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(first_row))

print("Contains correct solution =",any(np.array_equal(matrix[0,:], i) for i in first_row))

Solution took 0.0878 seconds
Number of poss solutions =  15
Contains correct solution = True


In [10]:
start = time.perf_counter()
second_row =[]
for row in first_row:
    solution[0,:] = row
    for split in splits:
        if possible(1,split,solution,correct):
            if column_check_top(1,split,solution,correct):
                second_row.append(np.array([row,split]))
                
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(second_row))
print("Contains correct solution =",any(np.array_equal(matrix[:2,:], i) for i in second_row))



Solution took 0.3115 seconds
Number of poss solutions =  14
Contains correct solution = True


In [11]:
third_row =[]
start = time.perf_counter()
for row1,row2  in second_row:
    solution = np.ones((9,9))*-1
    solution[0,:] = row1
    solution[1,:] = row2
    for split in splits:
            if possible(2,split,solution,correct):
                if column_check_top(2,split,solution,correct):
                    third_row.append([row1,row2,split])

stop =  time.perf_counter()   

print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(third_row))
print("Contains correct solution =",any(np.array_equal(matrix[:3,:], i) for i in third_row))

Solution took 0.3383 seconds
Number of poss solutions =  68
Contains correct solution = True


In [12]:
fourth_row =[]
start = time.perf_counter()
for row1,row2,row3  in third_row:
    solution = np.ones((9,9),dtype=int)*-1
    solution[0,:] = row1
    solution[1,:] = row2
    solution[2,:] = row3
    for split in splits:
            if possible(3,split,solution,correct):
                if column_check_top(3,split,solution,correct):
                    fourth_row.append([row1,row2,row3,split])
                    
stop =  time.perf_counter()   

print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(fourth_row))
print("Contains correct solution =",any(np.array_equal(matrix[:4,:], i) for i in fourth_row))

Solution took 0.7126 seconds
Number of poss solutions =  1164
Contains correct solution = True


In [13]:
fifth_row =[]
start = time.perf_counter()
for row1,row2,row3,row4  in fourth_row:
    solution = np.ones((9,9),dtype=int)*-1
    solution[0,:] = row1
    solution[1,:] = row2
    solution[2,:] = row3
    solution[3,:] = row4
    for split in splits:
            if possible(4,split,solution,correct):
                if column_check_top(4,split,solution,correct):
                    fifth_row.append([row1,row2,row3,row4,split])
                    
stop =  time.perf_counter()   

print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(fifth_row))
print("Contains correct solution =",any(np.array_equal(matrix[:5,:], i) for i in fifth_row))

Solution took 8.2064 seconds
Number of poss solutions =  5699
Contains correct solution = True


# alternate 

In [14]:
def column_check_bot(row,split,solution,y):
        temp = copy(solution)
        temp[row,:] = split       
        
        #check connectivuty
        if np.max(label(temp!=0,connectivity=1)) > 1 :
            #print("fail connect")
            return False
        
        temp *= y
        
        #check counts
        for j in range(1,10):
            if np.sum(temp == j) > j:
                #print("fail count",temp)
                return False
        
        # check that the top conditions are 
        for i in range(9):
            col = temp[:,i]
            if bot_labels[i] != 0:
                if split[i] == 1:
                    previous = np.sum(col[row+1:] !=0)
                    num = col[row]
                    if previous == 0:
                        if bot_labels[i] % num !=0:
                            #print("fail divisor")
                            return False
                    
                        remaining = bot_labels[i] / num
                        if remaining not in y[:row,i]:
                            #print("fail remaining. col = {} label = {} number= {} remaining = {} ".format(i,top_labels[i],num,remaining))
                            return False                    
                    
                    elif previous == 1:
                        if col[col!=0][-2:].prod() != bot_labels[i]:
                            #print("fail total")
                            return False                   
        return True      
    


In [15]:
start = time.perf_counter()
last_row =[]

solution = np.ones((9,9),dtype=int)*-1

for split in splits: 
    if possible(8,split,solution,correct):
        if column_check_bot(8,split,solution,correct):
            last_row.append(split)
            
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(last_row))

print("Contains correct solution =",any(np.array_equal(matrix[8,:], i) for i in last_row))

Solution took 0.0884 seconds
Number of poss solutions =  120
Contains correct solution = True


In [16]:
start = time.perf_counter()
eighth_row =[]
for row in last_row:
    solution = np.ones((9,9),dtype=int)*-1
    solution[8,:] = row
    for split in splits:
        if possible(7,split,solution,correct):
            if column_check_bot(7,split,solution,correct):
                eighth_row.append(np.array([split,row]))
                
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(eighth_row))
print("Contains correct solution =",any(np.array_equal(matrix[7:,:], i) for i in eighth_row))

Solution took 0.9626 seconds
Number of poss solutions =  124
Contains correct solution = True


In [17]:
start = time.perf_counter()
seventh_row =[]
for row1,row2 in eighth_row:
    solution = np.ones((9,9),dtype=int)*-1
    solution[7,:] = row1
    solution[8,:] = row2
    
    for split in splits:
        if possible(6,split,solution,correct):
            if column_check_bot(6,split,solution,correct):
                seventh_row.append(np.array([split,row1,row2]))
                
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(seventh_row))
print("Contains correct solution =",any(np.array_equal(matrix[6:,:], i) for i in seventh_row))

Solution took 0.8399 seconds
Number of poss solutions =  88
Contains correct solution = True


In [18]:
start = time.perf_counter()
sixth_row =[]
for row1,row2,row3 in seventh_row:
    solution = np.ones((9,9),dtype=int)*-1
    solution[6,:] = row1
    solution[7,:] = row2
    solution[8,:] = row3
    
    for split in splits:
        if possible(5,split,solution,correct):
            if column_check_bot(5,split,solution,correct):
                sixth_row.append(np.array([split,row1,row2,row3]))
                
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(seventh_row))
print("Contains correct solution =",any(np.array_equal(matrix[5:,:], i) for i in sixth_row))

Solution took 0.4899 seconds
Number of poss solutions =  88
Contains correct solution = True


In [34]:
start = time.perf_counter()
solved =[]
for row1,row2,row3,row4,row5 in fifth_row:
    for row6,row7,row8,row9 in sixth_row:
        solution=np.array([row1,row2,row3,row4,row5,row6,row7,row8,row9])
        if np.max(label(solution!=0,connectivity=1)) == 1:
            for i in range(2,10):
                if np.sum(solution*correct==i) == i:
                    solved.append(solution)
                
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
print("Number of poss solutions = ",len(solved))
print("Contains correct solution =",any(np.array_equal(matrix, i) for i in solved))          


Solution took 48.5006 seconds
Number of poss solutions =  92186
Contains correct solution = True


In [33]:
88*5699

501512

In [138]:
correct = np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],
                    [8, 6, 6, 6, 6, 6, 6, 7, 9],
                    [8, 5, 4, 3, 2, 1, 6, 7, 9],
                    [8, 5, 4, 3, 2, 2, 6, 7, 9],
                    [8, 5, 4, 3, 3, 3, 6, 7, 9],
                    [8, 5, 4, 4, 4, 4, 6, 7, 9],
                    [8, 5, 5, 5, 5, 5, 6, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]],dtype=int)

solution = np.ones((9,9),dtype=int)*-1

In [178]:
def forced(grid,hooks):
    grid[hooks==1] = 1
    changes = 1
    while changes >0:
        changes = 0
        for row,col in itertools.product(range(9),range(9)):
            if grid[row,col]==-1:
                test =grid*hooks 
                above = test[:row,col]
                below = test[row+1:,col]
                left = test[row,:col]
                right  =test[row,col+1:]
            
                before = [above,below,left,right]   
                after = [below,above,right,left]
                avail_after = [[abs(i) for i in j] for j in after]
                param =[top_labels[col],bot_labels[col],left_labels[row],right_labels[row]]
                count_before = [np.sum(i > 0) for i in before ]
                num = hooks[row,col]

                for i in range(4):
                    if (param[i] != 0) & (np.sum(before[i] < 0) ==0):
                        if count_before[i] ==0:
                            if param[i] % num  != 0:
                                grid[row,col]= 0
                                changes +=1
                            if param[i] / hooks[row,col] not in avail_after[i]:
                                grid[row,col]= 0
                                changes +=1
                                
                        
    
    return(grid)
        
        
        
            

In [180]:
solution = np.ones((9,9),dtype=int)*-1
start = time.perf_counter()
x = forced(solution,correct)
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds'.format((stop-start)))
x

Solution took 0.0122 seconds


array([[-1,  0,  0,  0, -1, -1,  0, -1, -1],
       [-1, -1, -1,  0, -1, -1,  0, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1,  0],
       [ 0,  0, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1,  0, -1, -1, -1],
       [-1, -1, -1, -1,  0,  0, -1, -1, -1]])

In [169]:
x = [0,0,0,-1]

0


<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/03/20180228_hooks_3_ans.png" width="300" height="400">